In [1]:
import torch
from torch import nn
from torchvision import models, datasets, transforms
import time
from tqdm.auto import tqdm
import os

Utilites

In [62]:
def set_requires_grad(model, value=False):
    for param in model.parameters():
        param.requires_grad = value

input_size = 224
batch_size = 4
normalize = transforms.Compose([
    transforms.Resize(input_size),
    transforms.CenterCrop(input_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

device = torch.device("cuda:0")
num_classes = 10

criterion = nn.CrossEntropyLoss()

def train_model(model, dataloaders, criterion, optimizer, num_epochs=3):
    since = time.time()

    acc_history = {'train': [], 'val': []}
    loss_history = {'train': [], 'val': []}

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            n_batches = len(dataloaders[phase])
            for inputs, labels in tqdm(dataloaders[phase], total=n_batches):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            loss_history[phase].append(epoch_loss)
            acc_history[phase].append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    return model, acc_history

def test_model(model, testloader, criterion):
    n_batches = len(testloader)
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in tqdm(testloader, total=n_batches):
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    loss = running_loss / len(testloader.dataset)
    acc = running_corrects.double() / len(testloader.dataset)
    print('Val Loss: {:.4f} Val Acc: {:.4f}'.format(loss, acc))
    return acc, loss

1. Transfer-learning on imagenette dataset

In [3]:
# Load model

resnet18_model = models.resnet18(pretrained=True)
num_ftrs = resnet18_model.fc.in_features
resnet18_model.fc = nn.Linear(num_ftrs, num_classes)

In [4]:
# Load dataset

# download dataset and unpack:
dataset_path = "imagenette2"
if not os.path.exists(dataset_path):
    !wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz
    !tar xvzf imagenette2.tgz

trainset_imagenette = datasets.ImageFolder(root='./%s/train/' % dataset_path, transform=normalize)
trainloader_imagenette = torch.utils.data.DataLoader(trainset_imagenette, batch_size=batch_size, shuffle=True, num_workers=2)

testset_imagenette = datasets.ImageFolder(root='./%s/val/' % dataset_path, transform=normalize)
testloader_imagenette = torch.utils.data.DataLoader(testset_imagenette, batch_size=batch_size, shuffle=False, num_workers=2)

loaders_imagenette = {'train': trainloader_imagenette, 'val': testloader_imagenette}

In [5]:
# train parameters
train_optimizer = torch.optim.SGD(resnet18_model.parameters(), lr=0.001, momentum=0.9)
resnet18_model = resnet18_model.to(device)

In [6]:
# train model
train_model(resnet18_model, loaders_imagenette, criterion, train_optimizer, num_epochs=5)

Epoch 0/4
----------

train Loss: 0.5270 Acc: 0.8423

val Loss: 0.2019 Acc: 0.9409

Epoch 1/4
----------

train Loss: 0.3383 Acc: 0.8975

val Loss: 0.2281 Acc: 0.9332

Epoch 2/4
----------

train Loss: 0.2381 Acc: 0.9251

val Loss: 0.1684 Acc: 0.9524

Epoch 3/4
----------

train Loss: 0.2126 Acc: 0.9383

val Loss: 0.1979 Acc: 0.9406

Epoch 4/4
----------

train Loss: 0.1451 Acc: 0.9591

val Loss: 0.2134 Acc: 0.9368

Training complete in 8m 21s


(ResNet(
   (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
   (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu): ReLU(inplace=True)
   (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   (layer1): Sequential(
     (0): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU(inplace=True)
       (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
     (1): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU

In [7]:
# save fc layer trained on imagenette
imagenette_fc = resnet18_model.fc
# save model
torch.save(resnet18_model, "./resnet18_model_imagenette.pth")

2. Fine-tunning on cifar10 dataset

In [8]:
trainset_cifar10 = datasets.CIFAR10(root='./data', train=True, download=True, transform=normalize)
trainloader_cifar10 = torch.utils.data.DataLoader(trainset_cifar10, batch_size=batch_size, shuffle=True, num_workers=2)

testset_cifar10 = datasets.CIFAR10(root='./data', train=False, download=True, transform=normalize)
testloader_cifar10 = torch.utils.data.DataLoader(testset_cifar10, batch_size=batch_size, shuffle=False, num_workers=2)

loaders_cifar10 = {'train': trainloader_cifar10, 'val': testloader_cifar10}

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [9]:
num_ftrs = resnet18_model.fc.in_features
set_requires_grad(resnet18_model, False)
resnet18_model.fc = nn.Linear(num_ftrs, num_classes)
resnet18_model = resnet18_model.to(device)

In [10]:
pretrain_optimizer = torch.optim.SGD(resnet18_model.fc.parameters(), lr=0.001, momentum=0.9)
train_model(resnet18_model, loaders_cifar10, criterion, pretrain_optimizer, num_epochs=2)
# Train
set_requires_grad(resnet18_model, True)
train_model(resnet18_model, loaders_cifar10, criterion, train_optimizer, num_epochs=1)

Epoch 0/1
----------

train Loss: 1.4583 Acc: 0.5099

val Loss: 1.0345 Acc: 0.6617

Epoch 1/1
----------

train Loss: 1.3517 Acc: 0.5567

val Loss: 1.0038 Acc: 0.6791

Training complete in 5m 1s
Epoch 0/0
----------

train Loss: 0.6340 Acc: 0.7917

val Loss: 0.2913 Acc: 0.9041

Training complete in 7m 5s


(ResNet(
   (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
   (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu): ReLU(inplace=True)
   (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   (layer1): Sequential(
     (0): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU(inplace=True)
       (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
     (1): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU

In [11]:
torch.save(resnet18_model, "./resnet18_model_cifar10.pth")

3. Fine-tunning on imagenette dataset (try to return imagenette accuracy)

In [57]:
imagenette_model = torch.load("./resnet18_model_imagenette.pth")
imagenette_fc = imagenette_model.fc
imagenette_model = None

cifar10_model = torch.load("./resnet18_model_cifar10.pth")

set_requires_grad(cifar10_model, False)
cifar10_model.fc = imagenette_fc
cifar10_model = cifar10_model.to(device)

In [59]:
# train fc layer on imagenette
pretrain_optimizer = torch.optim.SGD(cifar10_model.fc.parameters(), lr=0.001, momentum=0.9)
train_model(cifar10_model, loaders_imagenette, criterion, pretrain_optimizer, num_epochs=12)

Epoch 0/5
----------

train Loss: 0.5158 Acc: 0.8379

val Loss: 0.3678 Acc: 0.8838

Epoch 1/5
----------

train Loss: 0.5026 Acc: 0.8399

val Loss: 0.3737 Acc: 0.8808

Epoch 2/5
----------

train Loss: 0.4970 Acc: 0.8452

val Loss: 0.3726 Acc: 0.8818

Epoch 3/5
----------

train Loss: 0.4948 Acc: 0.8441

val Loss: 0.3893 Acc: 0.8782

Epoch 4/5
----------

train Loss: 0.4773 Acc: 0.8487

val Loss: 0.3915 Acc: 0.8780

Epoch 5/5
----------

train Loss: 0.4774 Acc: 0.8552

val Loss: 0.4059 Acc: 0.8731

Training complete in 7m 48s


(ResNet(
   (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
   (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu): ReLU(inplace=True)
   (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   (layer1): Sequential(
     (0): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU(inplace=True)
       (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
     (1): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU

In [63]:
# final test
cifar10_model.eval()
test_model(cifar10_model, testloader_imagenette, criterion)


Val Loss: 0.4059 Val Acc: 0.8731


(tensor(0.8731, device='cuda:0', dtype=torch.float64), 0.405892432237127)